In [0]:
# Camada Bronze - Types - Magic: The Gathering 
# Objetivo: Processo EL (Extract & Load) com nomenclatura padronizada
# Características: Extract da staging (S3/Parquet) -> Load na bronze (Unity Catalog/Delta) 
# Melhorias: Nomenclatura padronizada, colunas padronizadas, merge incremental robusto

# =============================================================================
# BIBLIOTECAS UTILIZADAS
# =============================================================================
import logging
from datetime import datetime, timedelta
from pyspark.sql.functions import *

# =============================================================================
# CONFIGURAÇÃO DE SEGREDOS
# =============================================================================
def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")

# =============================================================================
# CONFIGURAÇÕES GLOBAIS
# =============================================================================
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

CATALOG_NAME = get_secret("catalog_name")
SCHEMA_NAME = "bronze"
TABLE_NAME = "TB_BRONZE_TYPES"

S3_BUCKET = get_secret("s3_bucket")
S3_STAGE_PREFIX = get_secret("s3_stage_prefix", "magic_the_gathering/stage")
S3_BRONZE_PREFIX = get_secret("s3_bronze_prefix", "magic_the_gathering/bronze")
S3_STAGE_PATH = f"s3://{S3_BUCKET}/{S3_STAGE_PREFIX}"
S3_BRONZE_PATH = f"{S3_BUCKET}/{S3_BRONZE_PREFIX}"

print("Iniciando pipeline EL Unity com nomenclatura padronizada - TB_BRONZE_TYPES")
print("Dados de referência MTG - sem filtro temporal")

# =============================================================================
# MAPEAMENTO DE COLUNAS GOV
# =============================================================================
# Mapeamento das colunas originais para nomenclatura GOV (alinhado com Silver)
COLUMN_MAPPING = {
    # Identificação
    'type_name': 'NME_TYPE',
    'endpoint': 'NME_ENDPOINT',
    
    # Metadados
    'source': 'NME_SOURCE',
    'ingestion_timestamp': 'DT_INGESTION',
    
    # Datas
    'created_at': 'DT_CREATED',
    'updated_at': 'DT_UPDATED'
}



In [0]:
# =============================================================================
# FUNÇÕES UTILITÁRIAS
# =============================================================================
def setup_unity_catalog():
    # Configura o Unity Catalog e schema
    try:
        spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG_NAME}")
        spark.sql(f"USE CATALOG {CATALOG_NAME}")
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME}")
        spark.sql(f"USE SCHEMA {SCHEMA_NAME}")
        return True
    except Exception as e:
        print(f"Erro ao configurar Unity Catalog: {e}")
        return False

def extract_from_staging(table_name):
    # EXTRACT: Lê dados da camada staging
    try:
        # Para TB_BRONZE_TYPES, ler do arquivo types da staging
        if table_name == "TB_BRONZE_TYPES":
            stage_path = f"{S3_STAGE_PATH}/*_types.parquet"
        else:
            stage_path = f"{S3_STAGE_PATH}/*_{table_name}.parquet"
        df = spark.read.parquet(stage_path)
        print(f"Extraídos {df.count()} registros de staging para {table_name}")
        return df
    except Exception as e:
        print(f"Erro no EXTRACT de staging: {e}")
        return None

def apply_governance_naming(df):
    # Aplica nomenclatura GOV nas colunas
    try:
        available_columns = df.columns
        column_mapping = {}
        renamed_columns = []
        for original_col, gov_col in COLUMN_MAPPING.items():
            if original_col in available_columns:
                column_mapping[original_col] = gov_col
                renamed_columns.append((original_col, gov_col))
        for original_col, gov_col in column_mapping.items():
            df = df.withColumnRenamed(original_col, gov_col)
        if renamed_columns:
            print("Colunas renomeadas para governança:")
            for orig, gov in renamed_columns:
                print(f"  {orig} → {gov}")
        else:
            print("Nenhuma coluna renomeada para governança.")
        return df
    except Exception as e:
        print(f"Erro ao aplicar nomenclatura GOV: {e}")
        return df

def apply_temporal_filter(df, table_name):
    # Para dados de referência MTG, não aplicar filtro temporal
    if table_name == "TB_BRONZE_TYPES":
        print("Dados de referência MTG - filtro temporal não aplicado")
        return df
    return df

def check_delta_table_exists(delta_path):
    # Verifica se a tabela Delta existe
    try:
        # Verificar se _delta_log existe
        delta_log_path = f"{delta_path}/_delta_log"
        try:
            files = dbutils.fs.ls(delta_log_path)
            if files:
                # Tentar ler a tabela para verificar se tem dados
                df = spark.read.format("delta").load(delta_path)
                count = df.count()
                print(f"Tabela Delta existe com {count} registros")
                return True, count > 0
            else:
                return False, False
        except:
            return False, False
    except Exception as e:
        print(f"Erro ao verificar tabela Delta: {e}")
        return False, False

def get_delta_schema_for_merge(delta_path):
    # Obtém o schema da tabela Delta existente, excluindo colunas de particionamento
    try:
        df = spark.read.format("delta").load(delta_path)
        schema_fields = [field.name for field in df.schema.fields]
        
        # Excluir colunas de particionamento do schema para merge
        partition_columns = ['INGESTION_YEAR', 'INGESTION_MONTH']
        merge_schema = [col for col in schema_fields if col not in partition_columns]
        
        print(f"Schema completo da tabela Delta: {schema_fields}")
        print(f"Schema para merge (sem particionamento): {merge_schema}")
        return merge_schema
    except Exception as e:
        print(f"Erro ao obter schema Delta: {e}")
        return []

def prepare_dataframe_for_merge(df, table_name, target_schema=None):
    # Prepara DataFrame para merge, garantindo compatibilidade de schema
    if not df:
        return None
    try:
        # Aplicar nomenclatura GOV
        df = apply_governance_naming(df)

        # Lista de colunas padronizadas para types
        colunas_padrao = [
            'NME_TYPE', 'NME_ENDPOINT', 'DT_INGESTION', 'NME_SOURCE', 'INGESTION_YEAR', 'INGESTION_MONTH',
            'DT_CREATED', 'DT_UPDATED'
        ]
        # Remover colunas não padronizadas
        for col in df.columns:
            if col not in colunas_padrao:
                df = df.drop(col)
                print(f"Coluna '{col}' removida do DataFrame (não está na governança)")

        # Elimina duplicidade de NME_TYPE no DataFrame de origem
        total_before = df.count()
        df = df.dropDuplicates(['NME_TYPE'])
        total_after = df.count()
        print(f"Removidas {total_before - total_after} duplicatas baseadas em 'NME_TYPE'")

        # Aplicar filtro temporal (não aplicado para dados de referência)
        df = apply_temporal_filter(df, table_name)

        # Se temos schema de destino, filtrar colunas compatíveis
        if target_schema:
            available_columns = df.columns
            compatible_columns = [col for col in available_columns if col in target_schema]
            if not compatible_columns:
                print("Nenhuma coluna compatível encontrada para merge")
                return None
            print(f"Colunas compatíveis para merge: {compatible_columns}")
            df = df.select(compatible_columns)

        return df
    except Exception as e:
        print(f"Erro ao preparar DataFrame para merge: {e}")
        return None

def execute_merge_with_specific_columns(delta_table, merge_df, merge_columns):
    # Executa merge especificando exatamente quais colunas atualizar
    try:
        # Construir a condição de merge (types usa 'NME_TYPE' como chave)
        merge_condition = "bronze.NME_TYPE = novo.NME_TYPE"
        
        # Construir as ações de UPDATE especificando colunas
        update_actions = {}
        for col_name in merge_columns:
            if col_name != 'NME_TYPE':  # Não atualizar a chave de merge
                update_actions[col_name] = f"novo.{col_name}"
        
        # Construir as ações de INSERT especificando colunas
        insert_actions = {}
        for col_name in merge_columns:
            insert_actions[col_name] = f"novo.{col_name}"
        
        print(f"Executando merge com {len(update_actions)} colunas para UPDATE")
        print(f"Colunas para UPDATE: {list(update_actions.keys())}")
        
        # Executar merge com ações específicas
        delta_table.alias("bronze").merge(
            merge_df.alias("novo"),
            merge_condition
        ).whenMatchedUpdate(set=update_actions) \
         .whenNotMatchedInsert(values=insert_actions) \
         .execute()
        
        print("Merge Delta executado com sucesso")
        return True
    except Exception as e:
        print(f"Erro no merge Delta: {e}")
        return False

def check_unity_table_exists(full_table_name):
    # Verifica se a tabela Unity Catalog existe
    try:
        # Tentar fazer uma consulta simples na tabela
        test_query = f"SELECT 1 FROM {full_table_name} LIMIT 1"
        spark.sql(test_query)
        print(f"Tabela Unity Catalog '{full_table_name}' existe")
        return True
    except Exception as e:
        print(f"Tabela Unity Catalog '{full_table_name}' não existe ou não está acessível")
        return False

def create_or_update_unity_catalog(full_table_name, delta_path):
    # Cria ou atualiza tabela Unity Catalog preservando modificações existentes
    try:
        table_exists = check_unity_table_exists(full_table_name)
        
        if not table_exists:
            # Criar tabela pela primeira vez
            print(f"Criando tabela Unity Catalog: {full_table_name}")
            create_table_sql = f"""
            CREATE TABLE IF NOT EXISTS {full_table_name}
            USING DELTA
            LOCATION '{delta_path}'
            COMMENT 'Tabela bronze de referência de tipos de cards do Magic: The Gathering com nomenclatura padronizada'
            """
            spark.sql(create_table_sql)
            print(f"Tabela Unity Catalog criada: {full_table_name}")
        else:
            # Tabela já existe, apenas atualizar propriedades de pipeline (sem perder modificações)
            print(f"Tabela Unity Catalog já existe: {full_table_name}")
            print("Atualizando apenas propriedades de pipeline (preservando modificações existentes)")
        
        # Sempre atualizar propriedades de pipeline (não afeta modificações customizadas)
        try:
            spark.sql(f"""
            ALTER TABLE {full_table_name} SET TBLPROPERTIES (
                'bronze_layer' = 'true',
                'data_source' = 'mtg_api',
                'last_processing_date' = '{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}',
                'governance_applied' = 'true',
                'nomenclature_standard' = 'GOV',
                'merge_strategy' = 'incremental',
                'reference_data' = 'true',
                'temporal_filter' = 'none',
                'partitioning' = 'INGESTION_YEAR_MONTH'
            )
            """)
            print("Propriedades de pipeline atualizadas")
        except Exception as e:
            print(f"Erro ao atualizar propriedades: {e}")
        
        return True
    except Exception as e:
        print(f"Erro ao criar/atualizar Unity Catalog: {e}")
        return False

def load_to_bronze_unity_incremental_gov(df, table_name):
    # LOAD: Carrega dados na camada bronze (Unity + Incremental + GOV)
    if not df:
        return None
    try:
        delta_path = f"s3://{S3_BRONZE_PATH}/{table_name}"
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        from delta.tables import DeltaTable
        
        # Verificar se tabela Delta existe
        delta_exists, has_data = check_delta_table_exists(delta_path)
        
        if not delta_exists or not has_data:
            # Primeira criação com particionamento
            print("Criando tabela Delta pela primeira vez com nomenclatura GOV...")
            df_processed = prepare_dataframe_for_merge(df, table_name)
            
            # Adicionar colunas de particionamento baseadas em DT_INGESTION
            from pyspark.sql.functions import year, month, current_timestamp
            
            df_with_partition = df_processed.withColumn("DT_INGESTION", current_timestamp()) \
                                          .withColumn("INGESTION_YEAR", year(col("DT_INGESTION"))) \
                                          .withColumn("INGESTION_MONTH", month(col("DT_INGESTION")))
            
            print(f"Particionamento criado com DT_INGESTION")
            
            df_with_partition.write.format("delta") \
                   .mode("overwrite") \
                   .option("overwriteSchema", "true") \
                   .partitionBy("INGESTION_YEAR", "INGESTION_MONTH") \
                   .save(delta_path)
            print("Tabela Delta criada com sucesso")
        else:
            # Para merge, obter schema da tabela existente e preparar dados
            print("Tabela Delta existe, preparando para merge...")
            target_schema = get_delta_schema_for_merge(delta_path)
            
            # Preparar DataFrame para merge
            merge_df = prepare_dataframe_for_merge(df, table_name, target_schema)
            if not merge_df:
                print("Nenhum DataFrame compatível para merge")
                return None
            
            # Executar merge com colunas específicas
            print("Executando merge Delta...")
            delta_table = DeltaTable.forPath(spark, delta_path)
            
            # Usar função específica para merge com colunas definidas
            merge_success = execute_merge_with_specific_columns(delta_table, merge_df, target_schema)
            if not merge_success:
                print("Falha no merge Delta")
                return None

        # Criar ou atualizar tabela Unity Catalog (PRESERVANDO MODIFICAÇÕES)
        create_or_update_unity_catalog(full_table_name, delta_path)
        
        return df
    except Exception as e:
        print(f"Erro no LOAD para bronze {table_name}: {e}")
        return None

def process_el_unity_incremental_gov(table_name):
    # Executa o pipeline EL Unity + Incremental + GOV
    stage_df = extract_from_staging(table_name)
    if not stage_df:
        print(f"Falha no EXTRACT de staging para {table_name}")
        return None
    
    result_df = load_to_bronze_unity_incremental_gov(stage_df, table_name)
    return result_df

def query_bronze_unity(table_name):
    # Consulta na tabela bronze Unity Catalog
    try:
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        count_query = f"SELECT COUNT(*) as total FROM {full_table_name}"
        count_result = spark.sql(count_query)
        count_result.show()
        
    except Exception as e:
        print(f"Erro ao consultar tabela bronze: {e}")

def show_delta_info(table_name):
    # Mostra informações da tabela Delta
    try:
        delta_path = f"s3://{S3_BRONZE_PATH}/{table_name}"
        from delta.tables import DeltaTable
        delta_table = DeltaTable.forPath(spark, delta_path)
        history = delta_table.history()
        print(f"Versões Delta: {history.count()}")
            
    except Exception as e:
        print(f"Erro ao mostrar informações Delta: {e}")

def show_sample_data(table_name):
    # Mostra dados de exemplo da tabela
    try:
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        sample_query = f"SELECT * FROM {full_table_name} LIMIT 5"
        print("Dados de exemplo:")
        spark.sql(sample_query).show(truncate=False)
        
    except Exception as e:
        print(f"Erro ao mostrar dados de exemplo: {e}")

def show_governance_info():
    # Mostra informações sobre a governança aplicada
    print("=" * 60)
    print("INFORMAÇÕES DE GOVERNAÇA - TB_REF_TYPES")
    print("=" * 60)
    print(f"Nomenclatura GOV aplicada: {len(COLUMN_MAPPING)} colunas")
    print(f"Dados de referência: sem filtro temporal")
    print(f"Chave de merge: NME_TYPE")
    print("=" * 60)



In [0]:
# =============================================================================
# EXECUÇÃO PRINCIPAL
# =============================================================================

try:
    spark
except NameError:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.getOrCreate()

setup_success = setup_unity_catalog()
if not setup_success:
    raise Exception("Falha ao configurar Unity Catalog")

# Executar pipeline com tratamento de erro robusto
try:
    types_bronze_df = process_el_unity_incremental_gov("TB_BRONZE_TYPES")
    if types_bronze_df:
        print("Pipeline executado com sucesso")
        query_bronze_unity("TB_BRONZE_TYPES")
        show_delta_info("TB_BRONZE_TYPES")
        show_sample_data("TB_BRONZE_TYPES")
        show_governance_info()
    else:
        print("Falha no pipeline")
except Exception as e:
    print(f"Erro durante execução do pipeline: {e}")
    print("Falha no pipeline") 